In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
tf.keras.optimizers.Adamax(learning_rate=1e-2)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

from tensorflow.keras.layers import Dense, Input, ZeroPadding1D
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.losses import mse, binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
#from keras import objectives
from tensorflow.python.client import device_lib
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import math
import time
import h5py
import POD_FOM as POD_FOM
import POD_real as POD_real

In [ ]:
scalar = 'T'

mins = [0.8,0.8,0.8,0.05,0.8,1,1,1,1,1,1,1,20,150,0.1,0.1,0.001,0.001,0.1,0.1,0.1,0.1,1,1,7,0.5,0.5,2,5,5,53,30,65,10,10,10,10,1280,0.06,0]
maxs = [0.95,0.95,0.95,0.2,0.95,10,10,10,10,10,10,15,115,1000,1,1,0.1,0.1,1,1,1,1,5,5,12,1,1,7,10,10,58,36,70,15,15,15,15,1420,0.13,90]

snap_path = 'Training_output/14OD{0}.txt'
test_path = 'Test_output/14OV{0}.txt'

n_modes = 8 # number of the PC vectors

pod_p = open('POD_FOM/{0}/{1}'.format(1,scalar), 'r')
pod_p_lines = pod_p.readlines()
int_length = len(pod_p_lines)
orth_basis = np.zeros((int_length,n_modes))

# read PC vectors
for i in range(n_modes):
    pod_p = open('POD_FOM/{0}/{1}'.format(i+1,scalar), 'r')
    pod_p_lines = pod_p.readlines()
    pod_size = 0
    for j in range(int_length):
        pod_p_line = float(pod_p_lines[j])
        pod_size = pod_size + pod_p_line*pod_p_line

    for j in range(int_length):
        pod_p_line = float(pod_p_lines[j])
        orth_basis[j,i] =pod_p_line/math.sqrt(pod_size)

# read input parameters
parameters = open('training_input.dat','r')
parameters_lines = parameters.readlines()
n_parameters = len(parameters_lines[0].split())
n_snaps = len(parameters_lines)
X_p = np.zeros((n_snaps,n_parameters))
for k in range(n_snaps):
    parameters_line = parameters_lines[k].split()
    for kp in range(n_parameters):
        X_p[k][kp]=(float(parameters_line[kp])-mins[kp])/(maxs[kp]-mins[kp])

# make a parameter matrix  
X_train_full = X_p
X = X_train_full

print(X.shape)

# make a parameter matrix for prediction
v_parameters = open('test_input.dat','r')

v_parameters_lines = v_parameters.readlines()
n_valids = len(v_parameters_lines)

X_v = np.zeros((n_valids, n_parameters))
for k in range(n_valids):
    v_parameters_line = v_parameters_lines[k].split()
    for kp in range(n_parameters):
        X_v[k][kp] = (float(v_parameters_line[kp]) - mins[kp]) / (maxs[kp] - mins[kp])
X_test = X_v
print(X_test.shape)
        
# read original matrix
snaps = np.zeros((int_length,n_snaps))
for k in range(n_snaps):
    snap_p = open(snap_path.format(k+1,scalar),'r')
    snap_p_lines = snap_p.readlines()

    for j in range(int_length):
        snap_p_line = float(snap_p_lines[j])
        snaps[j,k] = snap_p_line        
        
# calculate the PC coefficients
alpha = np.zeros((n_modes,n_snaps))
for i in range(n_modes):
    for k in range(n_snaps):
        alpha[i][k] = np.inner(orth_basis[:,i],snaps[:,k])

# Transposition for training DNN
alpha = np.transpose(alpha)
alpha_norm = np.zeros((alpha.shape))

# normalize the PC coefficients
for k in range(n_snaps):
    for kp in range(n_modes):
        alpha_norm[k][kp] = ( alpha[k][kp] - np.min(alpha[:,kp])) / (np.max(alpha[:,kp]) - np.min(alpha[:,kp]))
        
coeff_dim = n_modes 
y_train_full = alpha_norm[:100]
y = y_train_full
print(y.shape)

X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
# Hyperparameter grid
hidden_layers_options = [2, 3, 4]  # Number of hidden layers
neurons_options = [40, 80, 160, 320, 640, 1280] # Number of hidden neurons
batch_size_options = [5, 10, 20]  # Batch sizes
learning_rate_options = [0.01, 0.001, 0.0001]  # Learning rates

In [ ]:
def build_model(hidden_layers, neurons, learning_rate):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    model.add(BatchNormalization())
    for _ in range(hidden_layers - 1):
        model.add(Dense(neurons, activation='relu'))
        model.add(BatchNormalization())
    model.add(Dense(coeff_dim))  # Output layer for regression
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [ ]:
"""
# Grid search
best_score = np.inf
best_params = {}

for hidden_layers in hidden_layers_options:
    for neurons in neurons_options:
        for batch_size in batch_size_options:
            for learning_rate in learning_rate_options:
                print(f"Testing model with {hidden_layers} hidden layers, {neurons} neurons, batch size {batch_size}, and learning rate {learning_rate}")
                model = build_model(hidden_layers, neurons, learning_rate)
                model.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=0)
                score = model.evaluate(X_val, y_val, verbose=0)
                print(f"Test MSE: {score}")

                if score < best_score:
                    best_score = score
                    best_params = {'hidden_layers': hidden_layers, 'neurons': neurons, 'batch_size': batch_size, 'learning_rate': learning_rate}

print(f"Best score (MSE): {best_score}")
print(f"Best parameters: {best_params}")
"""

In [ ]:
# Build and train the model with the optimal hyperparameters
best_params = {'hidden_layers': 2, 'neurons': 80, 'batch_size': 5, 'learning_rate': 0.01}

# k-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_losses = []

for train_index, val_index in kf.split(X):
    X_train_cv, X_val_cv = X[train_index], X[val_index]
    y_train_cv, y_val_cv = y[train_index], y[val_index]
    model = build_model(best_params['hidden_layers'], best_params['neurons'], best_params['learning_rate'])
    model.fit(X_train_cv, y_train_cv, epochs=50, batch_size=5, verbose=1)
    val_loss = model.evaluate(X_val_cv, y_val_cv, verbose=0)
    cv_losses.append(val_loss)

average_cv_loss = np.mean(cv_losses)
print(f"Average CV Loss: {average_cv_loss}")

In [ ]:
# Train the ENN
X_final_train = X
y_final_train = y
final_model = build_model(best_params['hidden_layers'], best_params['neurons'], best_params['learning_rate'])
final_model.summary()
                          
final_models = [build_model(best_params['hidden_layers'], best_params['neurons'], best_params['learning_rate']) for _ in range(50)]

for final_model in final_models:
    history = final_model.fit(
        X_final_train, y_final_train,
        batch_size=best_params['batch_size'],
        epochs=250, # Set to a large number, early stopping will determine the actual number of epochs
        verbose=1
    )
    print("network was trained")

In [ ]:
# real additional data
parameters_real = open('training_input_real.dat','r')
parameters_lines_real = parameters_real.readlines()
n_parameters_real = len(parameters_lines_real[0].split())
n_snaps_real = len(parameters_lines_real)
X_p_real = np.zeros((n_snaps_real,n_parameters_real))
for k in range(n_snaps_real):
    parameters_line_real = parameters_lines_real[k].split()
    for kp in range(n_parameters_real):
        X_p_real[k][kp]=(float(parameters_line_real[kp])-mins[kp])/(maxs[kp]-mins[kp])

# real data matrix  
X_train_real = X_p_real
print(X_train_real.shape)

In [ ]:
# make a parameter matrix for prediction
v_parameters_real = open('test_input_real.dat','r')

v_parameters_lines_real = v_parameters_real.readlines()
n_valids_real = len(v_parameters_lines_real)

X_v_real = np.zeros((n_valids_real, n_parameters_real))
for k in range(n_valids_real):
    v_parameters_line_real = v_parameters_lines_real[k].split()
    for kp in range(n_parameters_real):
        X_v_real[k][kp] = (float(v_parameters_line_real[kp]) - mins[kp]) / (maxs[kp] - mins[kp])
X_test_real = X_v_real
print(X_test_real.shape)

In [ ]:
snap_path_real = 'Training_output_real/14RD{0}.txt'
test_path_real = 'Test_output_real/14RV{0}.txt'

n_modes_real = 7

pod_p_real = open('POD_real/{0}/{1}'.format(1,scalar), 'r')
pod_p_lines_real = pod_p_real.readlines()
int_length_real = len(pod_p_lines_real)
orth_basis_real = np.zeros((int_length_real,n_modes_real))

for i in range(n_modes_real):
    pod_p_real = open('POD_real/{0}/{1}'.format(i+1,scalar), 'r')
    pod_p_lines_real = pod_p_real.readlines()
    pod_size_real = 0
    for j in range(int_length_real):
        pod_p_line_real = float(pod_p_lines_real[j])
        pod_size_real = pod_size_real + pod_p_line_real * pod_p_line_real

    for j in range(int_length_real):
        pod_p_line_real = float(pod_p_lines_real[j])
        orth_basis_real[j,i] =pod_p_line_real/math.sqrt(pod_size_real)

snaps_real = np.zeros((int_length_real,n_snaps_real))
for k in range(n_snaps_real):
    snap_p_real = open(snap_path_real.format(k+1,scalar),'r')
    snap_p_lines_real = snap_p_real.readlines()

    for j in range(int_length_real):
        snap_p_line_real = float(snap_p_lines_real[j])
        snaps_real[j,k] = snap_p_line_real        
        

alpha_real = np.zeros((n_modes_real,n_snaps_real))
for i in range(n_modes_real):
    for k in range(n_snaps_real):
        alpha_real[i][k] = np.inner(orth_basis_real[:,i],snaps_real[:,k])


alpha_real = np.transpose(alpha_real)
alpha_norm_real = np.zeros((alpha_real.shape))


for k in range(n_snaps_real):
    for kp in range(n_modes_real):
        alpha_norm_real[k][kp] = ( alpha_real[k][kp] - np.min(alpha_real[:,kp])) / (np.max(alpha_real[:,kp]) - np.min(alpha_real[:,kp]))
        
coeff_dim_real = n_modes_real 
y_train_real = alpha_norm_real
print(y_train_real.shape)

In [ ]:
# Function to create the transfer learning model
def create_transfer_model(pretrained_model):
    for layer in pretrained_model.layers:
        layer.trainable = False

    transfer_model = Sequential()
    transfer_model.add(pretrained_model)
    transfer_model.add(Dense(coeff_dim_real))
    transfer_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')
    return transfer_model

# Create and train the transfer learning model
transfer_model = create_transfer_model(final_model)
transfer_model.summary()

In [ ]:
transfer_models = [create_transfer_model(final_model) for _ in range(50)]

# Training each model
for transfer_model in transfer_models:
    history = transfer_model.fit(
        X_train_real, y_train_real,
        batch_size=best_params['batch_size'],
        epochs=200, # Set to a large number, early stopping will determine the actual number of epochs
        verbose=1
    )
    print("transfer network was trained")

In [ ]:
norm_p_alpha = np.mean([transfer_model.predict(X_test_real) for transfer_model in transfer_models], axis=0)

# Denormalize POD coefficient
p_alpha = np.zeros((norm_p_alpha.shape))
for kp in range(len(np.transpose(alpha_real))):
    p_alpha[:,kp] = norm_p_alpha[:,kp] * (np.max(alpha[:,kp]) - np.min(alpha[:,kp])) + np.min(alpha[:,kp])

if n_valids_real == 1:
    recons_p_field = np.matmul(basis,np.transpose(p_alpha))
    recons_p_field = recons_p_field.flatten()

else:
    recons_p_field = np.matmul(orth_basis_real,np.transpose(p_alpha))

sol = np.zeros((int_length_real, n_valids_real))

for kp in range(n_valids_real):
    for i in range(int_length_real):
        sol[i, kp] = recons_p_field[i, kp]

# build predicted fields
if not os.path.isdir('predict_TL'):
    os.mkdir('predict_TL')
for k in range(n_valids_real):
    print('Predict {0} for parameter{1}'.format(scalar, k + 1))
    if not os.path.isdir('predict_TL/{0}'.format(k + 1)):
        os.mkdir('predict_TL/{0}'.format(k + 1))
    reconst_p = open('predict_TL/{0}/{1}'.format(k + 1, scalar), 'w')

    if n_valids_real == 1:
        sol.flatten()
        for i in range(int_length):
            rec_p = sol[i]
            reconst_p.write(str(rec_p))
            reconst_p.write('\n')

    else:
        for i in range(int_length):
            rec_p = sol[i, k]
            reconst_p.write(str(rec_p))
            reconst_p.write('\n')
reconst_p.close()


In [ ]:
n_validation = 10

cfd_path = 'predict_TL/{0}/{1}'
val_path = 'Test_output_real/14RV{0}.txt'

int_length = 1666 #numbers of grid
scalar = 'T'

predict = np.zeros((int_length,n_validation))
diff = np.zeros((int_length,n_validation))
absol = np.zeros((int_length,n_validation))
errsqr = np.zeros((int_length,n_validation))
valsqr = np.zeros((int_length,n_validation))
valsqrsum = np.zeros((n_validation))
errsqrsum = np.zeros((n_validation))
val_mean = np.zeros((n_validation))
val_mean1 = np.ones((int_length, n_validation))
sqrsum = np.zeros((int_length,n_validation))
totsqrsum = np.zeros((n_validation))

snap_stack_val = np.zeros((int_length, n_validation))

for val_num in range(n_validation):
    predict_file = open(cfd_path.format(val_num+1,scalar),'r')
#   predict_file = open('predict/{0}/{1}'.format(val_num+1,scalar),'r')
    predict_file_lines = predict_file.readlines()
    snap_file = open(val_path.format(val_num+1,scalar),'r')
    snap_file_lines = snap_file.readlines()
    for i in range(int_length):
        predict_file_line = float(predict_file_lines[i])
        predict[i,val_num] = predict_file_line
        snap_file_line = float(snap_file_lines[i])
        snap_stack_val[i,val_num] = snap_file_line
        
#        diff[i,val_num] = snap_stack[i,val_num] - predict[i,val_num]
        diff[i,val_num] = snap_stack_val[i,val_num] - predict[i,val_num]
        absol[i,val_num] = abs(diff[i,val_num])
        errsqr[i,val_num] = diff[i,val_num] * diff[i,val_num]
#        valsqr[i,val_num] = snap_stack[i,val_num] * snap_stack[i,val_num]
        valsqr[i,val_num] = snap_stack_val[i,val_num] * snap_stack_val[i,val_num]
    errsqrsum[val_num] = np.sum(errsqr[:,val_num])
    valsqrsum[val_num] = np.sum(valsqr[:,val_num])
#    val_mean[val_num] = np.mean(snap_stack[:,val_num])
    val_mean[val_num] = np.mean(snap_stack_val[:,val_num])
    
    val_mean1[:,val_num] = val_mean1[:,val_num] * val_mean[val_num]
    for i in range(int_length):
#        sqrsum[i, val_num] = (snap_stack[i,val_num] - val_mean1[i,val_num]) * (snap_stack[i,val_num] - val_mean1[i,val_num])
        sqrsum[i, val_num] = (snap_stack_val[i,val_num] - val_mean1[i,val_num]) * (snap_stack_val[i,val_num] - val_mean1[i,val_num])

    totsqrsum[val_num] = np.sum(sqrsum[:,val_num])

rmse = np.zeros((1,n_validation))
nrmse = np.zeros((1,n_validation))
mae = np.zeros((1,n_validation))
mape = np.zeros((1,n_validation))
r2 = np.zeros((1,n_validation))
l2rel = np.zeros((1,n_validation))

for i in range(n_validation):
#    rmse[0,i] = round(math.sqrt(mean_squared_error(snap_stack[:,i],predict[:,i])),8)
    rmse[0,i] = round(math.sqrt(mean_squared_error(snap_stack_val[:,i],predict[:,i])),8)
#    nrmse[0,i] = round(rmse[0,i]*100/(np.mean(snap_stack[:,i])),8)
    nrmse[0,i] = round(rmse[0,i]*100/(np.mean(snap_stack_val[:,i])),8)
    #nrmse[0,i] = round(rmse[0,i]*100/(np.max(val[:,i])-np.min(val[:,i])),8)
#    mape[0,i] = round(100*np.sum(absol[:,i]/snap_stack[:,i])/n_mask,8)
    mape[0,i] = round(100*np.sum(absol[:,i]/snap_stack_val[:,i])/int_length,8)
    mae[0,i] = np.mean(absol[:,i])
    r2[0,i] =  1 - ( errsqrsum[i] / totsqrsum[i])
    l2rel[0,i] = math.sqrt( errsqrsum[i] / valsqrsum[i] )*100	

if not os.path.isdir('error_TL'):
    os.mkdir('error_TL')

err_file = open('error_TL/error_TL_{0}'.format(scalar),'w')
err_file.write('RMSE')
err_file.write('\t\t\t\t')
err_file.write('MAE')
err_file.write('\t\t\t\t')
err_file.write('MAX Error')
err_file.write('\t\t\t\t')
err_file.write('R2')
err_file.write('\t\t\t\t')
err_file.write('L2 relative norm')
err_file.write('\n')

for val_num in range(n_validation):
    
    print('Error check for validation {0}\n'.format(val_num+1))
    print('RMSE : {0}'.format(abs(rmse[0,val_num])))
    print('MAE : {0}'.format(abs(mae[0,val_num])))
    print('Max Error : {0}'.format(round(np.max(absol[:,val_num]),8)))
    print('R2 : {0}'.format(r2[0,val_num]))
    print('L2 relative norm : {0}\n'.format(l2rel[0,val_num]))

    err_file.write(str(rmse[0,val_num]))
    err_file.write('\t\t\t\t')
    err_file.write(str(mae[0,val_num]))
    err_file.write('\t\t\t\t')
    err_file.write(str(np.max(absol[:,val_num])))
    err_file.write('\t\t\t\t')
    err_file.write(str(r2[0,val_num]))
    err_file.write('\t\t\t\t')
    err_file.write(str(l2rel[0,val_num]))
    err_file.write('\n')

print('Average RMSE  : {0}%'.format(round(np.mean(abs(nrmse[0,:])),8)))
print('Average R2  : {0}'.format(round(np.mean(r2[0,:]),5)))
print('Average L2 relative norm  : {0}'.format(round(np.mean(l2rel[0,:]),5)))
err_file.close()
print("cpu time : ",  time.time()- stime)